In [1]:
%%javascript
    $("#homeButton").remove()
    $('body').append('<a href="#'+$("h1,h2,h3:eq(0)").attr("id")+'" style="position: fixed; bottom: 10px; right: 10px;" type="button" class="btn btn-success btn-circle btn-lg"><i class="glyphicon glyphicon-link" id="homeButton"></i></a>');
    $("#MainMenu").remove()
    var menu = 
      '<div id="MainMenu" style="position: fixed; top: 120px; right: 10px; z-index:6; max-height: 500px;">'+
        '<div class="list-group panel" >'+
          '<a href="#collapseMenu" class="list-group-item list-group-item-success" data-toggle="collapse" data-parent="#MainMenu">Indíce<i class="fa fa-caret-down"></i></a>'+
          '<div class="collapse" id="collapseMenu" style="overflow-y: overlay; max-height: 500px;">'+
          '</div>'+
        '</div>'+
      '</div>'
   
    var parent = $(menu)
    var arrayIds = []
    $("h1,h2,h3").attr("id",function(id,Value){
        if(Value != ""){
            var content = (Value.replace(new RegExp('-', 'g'), ' '));
            content = "&nbsp;".repeat(parseInt($(this).prop("tagName")[1])*2-1) + content;
            $(parent).find("#collapseMenu").append('<a href="#'+Value+'" style="position:relative;" class="list-group-item" data-parent="#SubMenu1">'+content+'</a>');
        }
    })
$('body').append(parent)

<IPython.core.display.Javascript object>

In [2]:
%%html
<style>
    h3{
        color:#0094E4;
    }
    h2{
         color:#F28773;
    }
    h1{
         color:#0094E4;
    }
</style>

<img src="./images/logo.png" style="width: 20%; margin-left: 70%;">
<img src="./images/python.png">

# DataFrames

En Spark 2.0 y otras versiones más recientes los DataFrames son el "caballito de batalla" y la forma principal de trabajar con Python y Spark en conjunto.

<div class="alert alert-info">Los <strong>DataFrames</strong> funcionan como tablas o DataFrames de pandas, ya que se comportan como si tuvieran filas y columnas. En realidad son filas distribuidas.</div>

Usar **DataFrames** proporciona varias ventajas:
* Sintaxis más simple que la de los RDDs
* Posibilidad de usar comandos tipo SQL directamente sobre el DataFrame
* Las operaciones están automáticamente distribuidas en los RDDs
 
La principal ventaja de usar los DataFrames de Spark sobre Pandas o cualquier otra cosa es que Spark puede manipular datos a través de muchos RDDs, cantidades masivas de datos que nocaben en una sola computadora. El costo por esto es una sintaxis un poquito rara, pero nada terrible.

## Para crear un DataFrame

Primero necesitamos importar una SparkSession:

In [ ]:
#Initiallizing Spark (just run once) https://jaceklaskowski.gitbooks.io/mastering-apache-spark/spark-sparkcontext-creating-instance-internals.html
import pyspark
sc = pyspark.SparkContext('local[*]')
sc

In [ ]:
from pyspark.sql import SparkSession


<div class="alert alert-info">La SparkSession nos permite accesar a todas las funcionalidades de Spark (incluyendo Data frames), mientras que el Spark Context no incluye DataFrames.</div>

Luego, inicializar la SparkSession

In [ ]:
# Puede que tome un tiempo en una computadora local
spark = SparkSession.builder.appName("Basics").getOrCreate()

Para crear un data frae necesitamos alguna de las siguientes opciones:
* Obtener datos de un archivo.
* Conectarnos a un archivo grande y distribuido (como HDFS).
* Convertir un RDD en DF

In [ ]:
#Vamos a empezar con cómo leer desde un archivo

df = spark.read.json('./data/people2.json')

## Mostrar los datos

In [ ]:
df.show()

In [ ]:
df.limit(10).toPandas()

In [ ]:
df.columns

In [ ]:
df.describe()

### ¿Cómo sé qué otros métodos hay para mi DataFrame?

In [ ]:
df

<div class="alert alert-danger" role="alert">
  Para DataFrames no tenemos <strong>isEmpty</strong> ni <strong>countApprox</strong>, por lo que si queremos conocer si contiene datos nuestroDataFrame es recomendable convertirlo en RDD primero. En general no es buena idea hacer un <strong>collect</strong> o un <strong>count</strong> en DataFrames cuya magnitud no conocemos.
</div>



In [ ]:
df.rdd.isEmpty()

### Esquema

En algunos tipos de datos como los formatos tabulares de csv, es muy fácil inferir un esquema, pero es posible que sea necesario definir el esquema manualmente si se planea utilizar un métodt .read que no tenga la capacidad de inferir un esquema (inferSchema).

In [ ]:
df.printSchema()

Spark cuenta con las herramientas para realizar lo anterior:

In [ ]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

Una vez que cargamos las herramientas necesitamos crear la lista de los campos estructurados (StructFields): 
* :param name: string, nombre del campo.
* :param dataType: :class:`DataType` del campo.
* :param nullable: boolean, Si puede ser nulo (None) o no.

In [ ]:
data_schema = [StructField("age", IntegerType(), True),StructField("name", StringType(), True)]

Definimos el esquema por completo

In [ ]:
final_struc = StructType(fields=data_schema)

Y por último asignamos el esquema 

In [ ]:
df = spark.read.json('people.json', schema=final_struc)

In [ ]:
df.printSchema()

## Seleccionar datos

In [ ]:
df['age']

In [ ]:
type(df['age'])

In [ ]:
df.select('age')

In [ ]:
type(df.select('age'))

In [ ]:
df.select('age').show()

In [ ]:
# Returns list of Row objects
df.head(2)

### Seleccionar múltiples columnas

In [ ]:
df.select(['age','name'])

In [ ]:
df.select(['age','name']).show()

## Crear nuevas columnas

In [ ]:
# Agregar una nueva columna que es una copia de otra
df.withColumn('newage',df['age']).show()

In [ ]:
df.show()

In [ ]:
# Renombrar una columna
df.withColumnRenamed('age','supernewage').show()

#### Algunas operaciones ligeramente más complicadas para agregar una columna 

In [ ]:
df.withColumn('doubleage',df['age']*2).show()

In [ ]:
df.withColumn('add_one_age',df['age']+1).show()

In [ ]:
df.withColumn('half_age',df['age']/2).show()

In [ ]:
df.withColumn('half_age',df['age']/2)

## Filtrado

In [ ]:
df = spark.read.csv('./data/appl_stock.csv', inferSchema=True, header=True)

In [ ]:
df.printSchema()

In [ ]:
df.limit(10).toPandas()

El filtrado funciona igual que en el caso de los RDDs, nos devuelve los datos que cumplan con cierta condición.

In [ ]:
df.filter(df.Close < 500).limit(5).toPandas()

In [ ]:
df.filter(df.Close < 500).describe().toPandas()

In [ ]:
#Otro modo de hacerlo
df.filter("Close < 500").limit(5).toPandas()

Puede ser que yo no quiera ver todo.

In [ ]:
df.filter("Close < 500").select(['Open','Close']).show()

Ok, entonces, ¿qué hacen los siguientes comandos de spark con nuestro DF original?

In [ ]:
df.filter((df['Close'] < 200) & (df['Open'] > 200)).limit(5).toPandas()

In [ ]:
df.filter((df['Close'] < 200) & ~(df['Open'] > 200)).limit(5).toPandas()

In [ ]:
df.filter((df['Low'] == 197.16)).toPandas()

### Podemos guardar información que nos interese particularmente en un diccionario:

In [ ]:
result = df.filter((df['Low'] == 197.16)).collect()

In [ ]:
row = result[0]

In [ ]:
row.asDict()

In [ ]:
row.asDict()['Volume']

## groupBy y aggregate
* groupBy: agrupa datos juntos con base en los valores de alguna columna
* aggregate: combines aggregates combina y/o agrega en una sola salida (como las sumas)

In [ ]:
df = spark.read.csv('./data/sales_info.csv',inferSchema=True,header=True)

In [ ]:
df.limit(6).show()

In [ ]:
df.printSchema()

In [ ]:
df.groupBy("Company").mean().show()

In [ ]:
df.groupBy("Company").sum().show()

In [ ]:
df.groupBy("Company").max().show()

### aggregate

<div class="alert alert-info"> Cuando no agregamos GroupBy, solamente estamos agregando a través de todo </div>

<div class="alert alert-info"> Una de las formas de agregar es usar un diccionario como entrada </div>

In [ ]:
df.agg({'Sales':'sum'}).show()

In [ ]:
df.agg({'Sales':'max'}).show()

#### Con grupo de agregación

In [ ]:
group_data = df.groupBy("Company")

In [ ]:
group_data.agg({'Sales':'max'}).show()

Hay más fucnciones de agregación como:

In [ ]:
from pyspark.sql.functions import countDistinct,avg,stddev

## Ejercicios

### Cargar el Walmart stock CSV File, con Saprk infiriendo el esquema

In [ ]:
df = spark.read.csv('./data/walmart_stock.csv',inferSchema=True,header=True)

### ¿Cómo se llaman las columnas?

### ¿Cómo se ve el esquema?

### ¿Cómo imprimirían las primeras cinco líneas?

### ¿Cómo usamos describe para aprender del DataFrame?

La media y la desviación estándar nos muestran muchooos puntos decimales. Podemos formatear los números para mostrarnos tan sólo hasta dos decimales. 

(Hint, primero hay que chear el esquema de describe)

In [ ]:
from pyspark.sql.functions import format_number
result = df.describe()
result2 = result.select(result['summary'],
              format_number(result['Open'].cast('float'),2).alias('Open'),
              format_number(result['High'].cast('float'),2).alias('High'),
              format_number(result['Low'].cast('float'),2).alias('Low'),
              format_number(result['Close'].cast('float'),2).alias('Close'),
              result['Volume'].cast('int').alias('Volume')
             )
result2.show()

### Crea un nuevo DataFrame con una columna que se llame HV Ratio que sea el cociente de el High Prices vs el volumen del stock (Volume).

In [ ]:
df.select(
    (df.High/df.Volume).alias("HV Ratio")
).show()

### ¿Qué día fue el pico más alto en High?

In [ ]:
# Didn't need to really do this much indexing
# Could have just shown the entire row
df.orderBy(df["High"].desc()).head()

In [ ]:
df.orderBy(df["High"].desc()).head(1)[0][0]

### ¿Cuál es la media de la columna Close?

In [ ]:
from pyspark.sql.functions import col as c
import pyspark.sql.functions as F

In [ ]:
df.select(
 F.avg(df["Close"].cast('Float')).cast('Float').alias("MEAN_Close")
).show()

In [ ]:
df.select(F.mean("Close")).show()

### ¿Cuáles son el máximo y el mínimo de la columna Volume?

In [ ]:
df.select(
    F.max("Volume"),
    F.min("Volume")
).show()

### ¿Cuál fue el máximo High al año?

In [ ]:
df.limit(3).toPandas()

In [ ]:
df2 = df.select('*',
          F.year("Date").alias('year'))

In [ ]:
df2.groupBy('year').max('High').show()

## Recursos
- [About Spark in mapr](https://mapr.com/blog/spark-101-what-it-what-it-does-and-why-it-matters/)
- [edX course Big Data analytics Using Spark](https://courses.edx.org/courses/course-v1:UCSanDiegoX+DSE230x+1T2018/courseware/2966295eefb4492b836de9ee34f05911/56608329f8fc42fc8fcaa5e49673ee3b/5?activate_block_id=block-v1%3AUCSanDiegoX%2BDSE230x%2B1T2018%2Btype%40vertical%2Bblock%4042ff48e505e24071b920a496b48a32bc)
- [paper regarding RDDs](https://www.usenix.org/system/files/conference/nsdi12/nsdi12-final138.pdf)
- [saurzcode on what is RDD](https://saurzcode.in/2015/10/what-is-rdd-in-spark-and-why-do-we-need-it/)
- [Spark-py notebooks by Jose A. Dianes](https://github.com/jadianes/spark-py-notebooks)
- [Presenting code using Jupyter Notebook Slides](https://medium.com/@mjspeck/presenting-code-using-jupyter-notebook-slides-a8a3c3b59d67)
- http://dme.rwth-aachen.de/en/research/projects/mapreduce
- https://softwareengineering.stackexchange.com/questions/98800/is-mapreduce-anything-more-than-just-an-application-of-divide-and-conquer
- http://data-flair.training/forums/topic/sparksession-vs-sparkcontext-in-apache-spark